In [ ]:
import gzip
import json
import re
import os
import sys
import time
import numpy as np
import pandas as pd
from itertools import islice
import psycopg2
from psycopg2.extras import execute_values
pd.options.display.float_format = '{:,}'.format

def batched(iterable, n):
    "Batch data into tuples of length n. The last batch may be shorter."
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while (batch := tuple(islice(it, n))):
        yield batch

# convert string to int or none
def parseInt(string):
    return int(float(string)) if string else None

In [ ]:
con = psycopg2.connect(
    host="localhost",
    database="reviews",
    user="postgres",
    password="postgres"
    )
cur = con.cursor()

con_lite = psycopg2.connect(
    host="localhost",
    database="reviews_lite",
    user="postgres",
    password="postgres"
    )
cur_lite = con_lite.cursor()

In [ ]:
cur_lite.execute('select id from review_embed')
review_ids = cur_lite.fetchall()

for review_id in review_ids:
    cur.execute("select * from review where id = %s", review_id)
    review = cur.fetchone()
    cur.execute('select * from book where id = %s' % review[1])
    book = cur.fetchone()

    cur.execute('select id, name from author left join authorForBook on author.id = authorForBook.authorId where bookId = %s' % book[0])
    authors = cur.fetchall()

    cur.execute('select * from authorForBook where bookId = %s' % book[0])
    authorsForBook = cur.fetchall()

    cur.execute('select * from work where id = %s' % book[1])
    work = cur.fetchone()

    cur_lite.execute('select * from work where id = %s' % work[0])
    work_lite = cur_lite.fetchone()

    if work_lite is None:
        cur_lite.execute('insert into work values (%s, %s, %s)', work)

    cur_lite.execute('select * from book where id = %s' % book[0])
    book_lite = cur_lite.fetchone()

    if book_lite is None:
        cur_lite.execute('insert into book values (%s, %s, %s, %s, %s, %s, %s, %s)', book)

        if book[7] and work[1] == '':
            cur_lite.execute('update work set title = %s where id = %s', (book[3], work[0]))

        for author in authors:
            cur_lite.execute('select * from author where id = %s' % author[0])
            author_lite = cur_lite.fetchone()

            if author_lite is None:
                cur_lite.execute('insert into author values (%s, %s)', author)

        for authorForBook in authorsForBook:
            cur_lite.execute('insert into authorForBook values (%s, %s, %s)', authorForBook)
con_lite.commit()

In [ ]:
cur_lite.execute('select id from review_embed')
review_ids = cur_lite.fetchall()
for review_id in review_ids:

    cur.execute("select * from review where id = %s", review_id)
    review = cur.fetchone()
    cur.execute('select * from book where id = %s' % review[1])
    book = cur.fetchone()

    cur_lite.execute('insert into review values (%s, %s)', (review_id, book[0]))

con_lite.commit()

In [ ]:
from sentence_transformers import SentenceTransformer
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

In [ ]:
query_embedding = model.encode('modern sci-fi with a focus on big interesting ideas')
cur_lite.execute('''
            select book.title
            from review_embed
            inner join review on review_embed.id = review.id
            inner join book on review.bookId = book.id
            order by embedding <=> '%s'
            limit 15
''' % query_embedding.tolist())

df = pd.DataFrame(cur_lite.fetchall(), columns=['title'])

In [ ]:
df

In [ ]:
con_lite.commit()